Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД.
Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [1]:
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains as ac
from selenium.webdriver.chrome.options import Options
import json
from pprint import pprint
import pandas as pd

In [2]:
list_leters=[]
driver=webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get('https://www.mvideo.ru/')
pages=1
while True:
    try:
        button = WebDriverWait(driver,10).until(ec.presence_of_element_located((By.XPATH,'//div[@data-init="ajax-category-carousel"][2]//a[@class="next-btn sel-hits-button-next"]')))
        button.click()
        pages+=1
    except Exception as e:
        print(e,f'всего страниц {pages}')
        break

Message: 
 всего страниц 6


In [4]:
bloks= driver.find_elements_by_xpath("//div[@data-sel='new_cart-carousel-gallery_accessories']")
goods=bloks[1].find_elements_by_xpath(".//li[@class='gallery-list-item height-ready']//h4/a[@data-product-info]")

In [5]:
client = MongoClient('localhost:27017')
db = client.goods
collection = db.mvideo
collection.drop()
for i in goods:
    collection.insert_one(json.loads(i.get_attribute('data-product-info')))

In [6]:
for document in db.mvideo.find():
    pprint (document)

{'Location': 'block5260655',
 '_id': ObjectId('5f218c4017562fdb699cc8d4'),
 'eventPosition': 5,
 'productCategoryId': 'cat2_cis_0000000212',
 'productCategoryName': 'Весы',
 'productGroupId': 'cat1_cis_0000000136',
 'productId': '20064625',
 'productName': 'Весы напольные Tefal Body Up BM2520V0',
 'productPriceLocal': '3990.00',
 'productVendorName': 'Tefal'}
{'Location': 'block5260655',
 '_id': ObjectId('5f218c4017562fdb699cc8d5'),
 'eventPosition': 6,
 'productCategoryId': 'cat2_cis_0000000357',
 'productCategoryName': 'Смартфоны',
 'productGroupId': 'cat1_cis_0000000012',
 'productId': '30047892',
 'productName': 'Смартфон Samsung Galaxy A51 64GB White (SM-A515F)',
 'productPriceLocal': '19990.00',
 'productVendorName': 'Samsung'}
{'Location': 'block5260655',
 '_id': ObjectId('5f218c4017562fdb699cc8d6'),
 'eventPosition': 7,
 'productCategoryId': 'cat2_cis_0000000001',
 'productCategoryName': 'Телевизоры',
 'productGroupId': 'cat1_cis_0000000001',
 'productId': '10020821',
 'product

In [7]:
df = pd.DataFrame(db.mvideo.find())
print(df[['productName','productPriceLocal']])

                                          productName productPriceLocal
0               Весы напольные Tefal Body Up BM2520V0           3990.00
1   Смартфон Samsung Galaxy A51 64GB White (SM-A515F)          19990.00
2                         Телевизор Daewoo L24V639VAE           5990.00
3     Карта памяти SDHC Micro IMO 32GB Сlass 10 UHS-I            590.00
4   Смартфон Huawei P40 Lite Midnight Black (JNY-LX1)          19990.00
5             Микроволновая печь соло Gorenje MO17E1W           2990.00
6   Наушники Apple AirPods w/Charging Case (MV7N2R...          11490.00
7   Карта памяти SDXC Micro SanDisk Ultra 64GB (SD...           1690.00
8                     Электрочайник Moulinex BY730132           1990.00
9   Смартфон Samsung Galaxy A51 64GB Black (SM-A515F)          19990.00
10      Умные весы Xiaomi Mi Body Composition Scale 2           2490.00
11  Машинка для стрижки волос Rowenta Driver TN1600F0           1190.00
12           Беспроводная акустика JBL Charge 4 Black           

In [8]:
driver.close()